In [125]:
import requests
from bs4 import BeautifulSoup


def scraping(table):
    assert table != None
    data = []
    for row in table.find_all('tr'):
        columns = row.find_all('td')
        if len(columns) >= 7:
            time = columns[0].text.strip()
            temperature = columns[1].text.strip()
            dew_point = columns[2].text.strip()
            humidity = columns[3].text.strip()
            wind_speed = columns[4].text.strip()
            pressure = columns[5].text.strip()
            precipitation = columns[6].text.strip()

            # Create a tuple for each day's observations and add it to the data list
            daily_observation = (time, temperature, dew_point, humidity, wind_speed, pressure, precipitation)
            data.append(daily_observation)
    return data



def find_table(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find('table', class_='tb7')
        return table
    else :
        print("Failed to retrieve the webpage")

In [126]:
import calendar

months = list(calendar.month_name)[1:]
years = [ '2015', '2016' , '2017' , '2018'  , '2019' ,'2020' ]


URL = 'https://tcktcktck.org/dhi-qar/{month}-{year}'
URLs = []
for year in years:
    for month in months:
        _ = URL.format(month = month , year= year)
        URLs.append( _ )

In [127]:
len(URLs)

72

In [165]:
import pandas as pd
import numpy as np

def split(text):
    text = text
    text = text.split()
    return float(text[0])

def data_frame(data):
    df = pd.DataFrame(data[1:] )
    df2 = pd.DataFrame([] , columns = ['date' , 'C','Kph' , 'Hg'])
    df2['date'] = df[0]
    df2['C'] = df[1].apply(split)
    df2['Kph'] = df[4].apply(split)
    df2['Hg'] = df[5].apply(split)
    return df2

In [174]:
def collect():
    for i,url in enumerate(URLs):
        tabel = find_table(url)
        data = scraping(tabel)
        df = data_frame(data)
        df.to_csv(f'Dhi_Qar-{i}-.csv', index=False)
        print(i)
collect()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71


In [177]:
files = [f'Dhi_Qar-{i}-.csv' for i in range(72)]

def combine_csv():
    df1 = pd.read_csv(files[0])

    for file in files[1:]:
        df2 = pd.read_csv(file)
        df1 = pd.concat([df1, df2], axis=0)
    return df1

In [185]:
df = combine_csv()
df.to_csv(f'Dhi_Qar-Temp.csv', index=False)

In [186]:
df = pd.read_csv('Dhi_Qar-Temp.csv')

In [187]:
df.head()

,date,C,Kph,Hg
0,2015-01-01,16.0,8.0,30.15
1,2015-01-02,18.0,11.0,30.09
2,2015-01-03,18.0,7.0,30.06
3,2015-01-04,17.0,12.0,30.00
4,2015-01-05,16.0,11.0,30.09


In [188]:
df.shape

(2192, 4)

In [189]:
df.isnull().sum()

date    0
C       0
Kph     0
Hg      0
dtype: int64